# Data preprocess

In [ ]:
from expenses_preprocess import *
import os

DATA_PATH = "./data"
PERSON_WHO_PAYS = "Lucas Alencar"

# Map with categories from Nubank/Splitwise to official categories
CATEGORY_CONVERSION_TABLE = {
    'Eletrônicos': 'Compras',
    'Supermercado': 'Mercado',
    'Vestuário': 'Compras',
    'Jantar fora': 'Restaurante',
    'Ônibus/trem': 'Transporte',
    'Filmes': 'Lazer',
    'Despesas médicas': 'Saúde',
    'Aluguel': 'Casa',
    'Táxi': 'Transporte'
}

# Column names to convert Splitwise
SPLITWISE_COLUMN_NAMES = {
    'Data': 'date',
    'Descrição': 'title',
    'Categoria': 'category',
}

# NUBANK PREPROCESS
nubank_expenses = nubank_preprocess(read_all_csvs(DATA_PATH, 'nubank*.csv'))

# SPLITWISE PREPROCESS
splitwise = read_first_file_found(DATA_PATH, "Splitwise*.csv")
splitwise = splitwise_preprocess(splitwise, PERSON_WHO_PAYS, SPLITWISE_COLUMN_NAMES)

# OTHER EXPENSES PREPROCESS
other_expenses = read_first_file_found(DATA_PATH, "Outras*contas*Despesas*.csv")

# INCOMES PREPROCESS
incomes = read_first_file_found(DATA_PATH, "Outras*contas*Rendimentos*.csv")
incomes = incomes_preprocess(incomes)

# Data consolidation and exports
expenses = expenses_preprocess(pd.concat([nubank_expenses, splitwise, other_expenses]), CATEGORY_CONVERSION_TABLE)
export_to_sheets(expenses, DATA_PATH)
export_raw_expenses(expenses, DATA_PATH)

print("Ok")

# Monthly Expenses

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
from IPython.display import display

%matplotlib inline

def amount_by_month(data):
    return data.groupby(data['date'].dt.strftime("%Y-%m")).amount.sum().to_frame()

perc_format = "{:.2%}"
br_currency_format = "R$ {:.2f}"

MONTHLY_COST_COLS_FORMAT = {
    'Expenses': br_currency_format, 
    'Incomes': br_currency_format, 
    'Balance': br_currency_format, 
    'Balance (%)': perc_format
}

MONTHLY_BALANCE_GOAL = 0.25 # 25%

def balance(expense, income):
    balance = income + expense
    balance_perc = balance / income
    return [balance, balance_perc]

def summary_expenses(expenses, incomes):
    balance_val, balance_perc = balance(expenses, incomes)
    summary_exp = pd.concat([expenses, incomes, balance_val, balance_perc], axis=1).dropna()
    summary_exp.columns = ['Expenses', 'Incomes', 'Balance', 'Balance (%)']
    return summary_exp

# Expenses and incomes by month
exp_by_month = amount_by_month(expenses)
inc_by_month = amount_by_month(incomes)
monthly_exp = summary_expenses(exp_by_month, inc_by_month)

def amount_color(value):
    color = 'red' if value <= 0 else 'green'
    return "color: %s" % color

def balance_bg_color(mid_range):
    def bg(value):
        if value <= 0: color = 'red'
        elif value <= mid_range: color = 'yellow'
        else: color = 'green'
        return 'background-color: %s' % color
    return bg

monthly_exp_styled = monthly_exp.style\
    .format(MONTHLY_COST_COLS_FORMAT)\
    .applymap(amount_color, subset=['Expenses', 'Incomes'])\
    .applymap(balance_bg_color(MONTHLY_BALANCE_GOAL), subset=['Balance (%)'])

display(monthly_exp_styled)

# Monthly averages
avg_monthly_exp = summary_expenses(exp_by_month.mean(), inc_by_month.mean())
avg_monthly_exp.style.applymap(balance_bg_color(MONTHLY_BALANCE_GOAL), subset=['Balance (%)'])